In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy.stats as sstat
from mpmath import mp
import math
import random

psd = pd.DataFrame
from tensorflow_probability.substrates import numpy as tfp
tfd = tfp.distributions

In [2]:
def single_lik(mu):
   y= tfd.MultivariateNormalFullCovariance(mu,sigma_obs)
   return y

In [ ]:
## mu_obs=np.array([1,4])
## mu1=[1,4]
## sigma_obs=np.array([[1,0.5],[0.5,1]])
## sigma1=[[1, 0.5],[0.5,1]]
## y_obs=tfd.MultivariateNormalFullCovariance(mu_obs,sigma_obs).sample(1000)
## y_obs

In [3]:
def prior():
    return tfd.MultivariateNormalFullCovariance(mu0,sigma0)

In [4]:
def proposal(theta):
  return tfd.MultivariateNormalFullCovariance(theta,cov)

In [37]:
def K_h(u,H):
    return 1/(np.sqrt(2*math.pi))*pow(np.linalg.det(H), -1/2)*np.exp(-1/2*u@np.linalg.inv(H)@u.T)

In [31]:
u= np.array([2,1])
H= np.array([[1,0],[0,1]])
K_h(u,H)

0.032747176537766653

In [ ]:
def S1(y):
  quantiles=[]
  for i in range(9):
      quantiles.append(np.quantile(y,0.1+0.1*i))         #quantili 0,1 0,2 0,3 0,4 0,5 0,6 0,7 0,8 0,9
  return quantiles

In [16]:
def S2(y):
  return np.mean(y,axis=0)


In [17]:
mu_obs = np.array ([[1,4]])
sigma_obs=np.array([[1,0.5],[0.5,1]])
y_obs=tfd.MultivariateNormalFullCovariance(mu_obs,sigma_obs).sample(1000)
np.mean(y_obs,axis=0)

array([[1.00771218, 4.00604116]])

In [ ]:
S2(y_obs)

AttributeError: ignored

In [38]:
s_0 = S2(single_lik(prior().sample()).sample(1000))
s_obs = S2(y_obs)
print(s_0.shape, s_obs.shape)
(np.array(s_0)-np.array(s_obs)).shape

K_h(np.array(s_0)-np.array(s_obs),H)

(1, 2) (1, 2)


array([[4.56159465e-05]])

In [ ]:
def abc_algo (N, S, y_obs, H):
    s_obs = S(y_obs)
    theta=[]
    s = []
    y = []
    theta_0 = prior().sample()
    y_0 = single_lik(theta_0).sample() 
    s_0 = S(y_0)
    
    print('prima')
    while ((K_h(np.linalg.norm(np.array(s_0)-np.array(s_obs)),H)[0]<=np.array([0,0])[0]) and ( K_h(np.linalg.norm(np.array(s_0)-np.array(s_obs)),H)[1]<=np.array([0,0])[1])):
        print(K_h(np.linalg.norm(s_0-s_obs),H))
        theta_0 = prior().sample()
        y_0 = single_lik(theta_0).sample() 
        s_0 = S(y_0)
    print('dopo')
      
    theta.append(theta_0)
    s.append(s_0)
    y.append(y_0)
    theta_old = theta_0
    s_old = s_0
    y_old = y_0

    for i in range(N):
        theta_current = proposal(theta_old).sample()
        y_current = single_lik(theta_current).sample(1000)
        #y.append(y_current)
        s_current = S(y_current)
               
        prob = min([1.,(K_h(np.linalg.norm(s_current - s_obs),H)@prior().prob(theta_current)@proposal(theta_old).prob(theta_current))/(
             K_h(np.linalg.norm(s_old-s_obs),H)@prior().prob(theta_old)@proposal(theta_current).prob(theta_old))])
        u = np.random.uniform(low=0,high=1,size=2)
        if ((u[0] < prob[0]) and (u[1] < prob[1])):
              theta_i = theta_current
              s_i = s_current
        else: 
              theta_i = theta_old
              s_i = s_old
              #y.pop()
              #y.append(y_old)

        theta_old = theta_i
        s_old = s_i
        
        theta.append(theta_i)
        s.append(s_i)
        
    return  theta

In [48]:
def abc_algo (N, S, y_obs, H):
    s_obs = S(y_obs)
    theta=[]
    s = []
    y = []
    theta_0 = prior().sample()
    y_0 = single_lik(theta_0).sample() 
    s_0 = S(y_0)
    
    print('prima')
    while ((K_h(np.array(s_0)-np.array(s_obs),H)[0]<=np.array([0,0])[0]) and ( K_h(np.array(s_0)-np.array(s_obs),H)[1]<=np.array([0,0])[1])):
        print(K_h(s_0-s_obs,H))
        theta_0 = prior().sample()
        y_0 = single_lik(theta_0).sample() 
        s_0 = S(y_0)
    print('dopo')
      
    theta.append(theta_0)
    s.append(s_0)
    y.append(y_0)
    theta_old = theta_0
    s_old = s_0
    y_old = y_0

    for i in range(N):
        theta_current = proposal(theta_old).sample()
        y_current = single_lik(theta_current).sample(1000)
        #y.append(y_current)
        s_current = S(y_current)
               
        prob = min([1.,(K_h(np.array(s_current) - np.array(s_obs),H)*prior().prob(theta_current)*proposal(theta_old).prob(theta_current))/(
             K_h(np.array(s_old-s_obs),H)*prior().prob(theta_old)*proposal(theta_current).prob(theta_old))])
        ## u = np.random.uniform(low=0,high=1,size=2)
        u = tfd.Uniform(0,1).sample()
        if u < prob:
              theta_i = theta_current
              s_i = s_current
        else: 
              theta_i = theta_old
              s_i = s_old
              #y.pop()
              #y.append(y_old)

        theta_old = theta_i
        s_old = s_i
        
        theta.append(theta_i)
        s.append(s_i)
        
    return  theta

In [47]:
u = np.random.uniform(low=0,high=1,size=2)
u

array([0.1015613 , 0.19522931])

In [40]:
K_h(np.array(s_0)-np.array(s_obs),H)

array([[4.56159465e-05]])

In [ ]:
np.random.uniform(low=0,high=1,size=2)

array([0.2362373 , 0.20395123])

In [ ]:
prior().prob(proposal(prior().sample()).sample())@proposal(prior().sample()).prob(proposal(prior().sample()).sample())

1.338803565385489e-06

In [26]:
##VARIABILI:
cov= np.array([[1,0],[0,1]])
H= np.array([[1,0],[0,1]])
mu0 = np.array ([[3,6]])
sigma0 = np.array([[3,0.7],[0.7,3]])
n = 1000
mu_obs = np.array ([[1,4]])
sigma_obs=np.array([[1,0.5],[0.5,1]])
y_obs=tfd.MultivariateNormalFullCovariance(mu_obs,sigma_obs).sample(1000)

In [49]:
prova = abc_algo (n, S2, y_obs, H) 

prima
dopo


In [51]:
prova

[array([[5.59907809, 9.04630771]]),
 array([[4.67106847, 7.47536403]]),
 array([[4.67106847, 7.47536403]]),
 array([[4.18762508, 6.04591486]]),
 array([[4.32724447, 5.90520922]]),
 array([[4.42724817, 4.65041761]]),
 array([[3.99758494, 5.29243179]]),
 array([[3.26089815, 5.51516318]]),
 array([[3.73493274, 5.7299146 ]]),
 array([[3.73493274, 5.7299146 ]]),
 array([[1.92633673, 5.09707606]]),
 array([[2.5905935 , 5.13533975]]),
 array([[2.5905935 , 5.13533975]]),
 array([[2.576436 , 4.9377169]]),
 array([[1.21036755, 4.64671856]]),
 array([[1.21036755, 4.64671856]]),
 array([[0.80160574, 4.52786706]]),
 array([[1.47693822, 4.76425406]]),
 array([[1.47693822, 4.76425406]]),
 array([[1.6937088 , 4.32674982]]),
 array([[1.6937088 , 4.32674982]]),
 array([[0.9862103, 4.5759378]]),
 array([[0.9862103, 4.5759378]]),
 array([[0.9862103, 4.5759378]]),
 array([[2.32618209, 4.12367993]]),
 array([[2.32618209, 4.12367993]]),
 array([[1.18071334, 3.86738862]]),
 array([[1.18071334, 3.86738862]]),


In [56]:
prova=np.asarray(prova)
plt.scatter(prova[:,0], prova[:,1])
plt.axis('equal')
plt.show()

IndexError: ignored